In [1]:
import sys
import re
import pandas as pd
import json
from wordcloud import WordCloud
from os import path
from collections import Counter
import matplotlib.pyplot as plt
import jieba
import numpy as np

In [2]:
data = pd.read_excel('./data/data.xlsx')
data

,target,userId,sellerId,timestamp,comment
0,1,127,1708,2019-10-01 00:57:45,text：订餐1个半小时不见餐，催了还挂电话！☆ ☆ ☆
1,1,1661,886,2019-10-01 00:42:07,text：太不靠谱！× × 不是漏发订单就是菜送错了，要不就早送一小时！！！× ×
2,0,92758,10612,2019-10-01 00:51:18,text：超级超级喜欢吃，每次去都好开心，口味很好，建议手套多戴几双，吃完之后的形象是不太好...
3,0,1639,1699,2019-10-03 00:01:56,text：谢谢快递哥哥，非常准时，辛苦啦
4,1,241839,36556,2019-10-01 00:07:56,非常差得餐厅，上菜非常慢，且贵！属于可以在餐厅饿死的那种\n，面条没有味道，一碗28，坑死爹...
...,...,...,...,...,...
17948,0,34025,24833,2020-09-25 23:15:36,"text：感觉这里好像发挥不是很稳定,来了几此,有好有坏.吃日料主要就关注三文鱼,这里的属于..."
17949,1,656,901,2020-09-25 23:17:49,text：订的九点送到，十二点才到，呵呵× × × × × × × ×
17950,1,612,743,2020-09-27 23:36:59,text：差评！辣白菜五花肉石锅拌饭超级难吃！五花肉又硬又难吃，感觉不新鲜。以后再也不会吃这家店了
17951,1,146739,15687,2020-09-27 23:53:15,又贵又不好吃，老婆几次非要来看看，这次一吃真难吃。\n蒸没错，但比炒菜更油腻，没啥菜好吃，以...


In [3]:
def clean_data(x):
    text = re.sub(r"[A-Za-z0-9(),!?\'\'\.【】⊙﹏…ฅω*ฅ❀༵〜⚘;%^:#＆@？！‘’：_，“”、-。~～·\.-×☆]"," ",x)
    return text.strip()
data['comment']=data['comment'].apply(lambda x:clean_data(x).replace('\n',''))

In [4]:
stopword=[line.strip() for line in open('ChineseStopWords.txt','r',encoding="utf-8").readlines()]
def cut_word(text):
    try:
        clean_text=[]
        for i in jieba.lcut(text):
            if i:
                if i not in stopword and i:
                    clean_text.append(i)
        return ' '.join(clean_text)
    except:
        print(text)
data['comment']=data['comment'].apply(lambda x:cut_word(x))

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\inner\AppData\Local\Temp\jieba.cache
Loading model cost 0.571 seconds.
Prefix dict has been built successfully.


In [5]:
data.to_excel('clean_data.xlsx',index=None)

In [6]:
data = pd.read_excel('clean_data.xlsx')
data

,target,userId,sellerId,timestamp,comment
0,1,127,1708,2019-10-01 00:57:45,订餐 半小时 不见 餐 催 挂 电话
1,1,1661,886,2019-10-01 00:42:07,太 不靠 谱 漏发 订单 菜 送错 早 送 小时
2,0,92758,10612,2019-10-01 00:51:18,超级 超级 喜欢 吃 每次 好开心 口味 建议 手套 戴 几双 吃 完 形象...
3,0,1639,1699,2019-10-03 00:01:56,谢谢 快递 哥哥 准时 辛苦
4,1,241839,36556,2019-10-01 00:07:56,差得 餐厅 上菜 慢 贵 餐厅 饿死 那种 面条 味道 一碗 ...
...,...,...,...,...,...
17948,0,34025,24833,2020-09-25 23:15:36,感觉 好像 发挥 稳定 几此 有好有坏 吃 日料 关注 三文鱼 中等 不...
17949,1,656,901,2020-09-25 23:17:49,订 九点 送到 十二点
17950,1,612,743,2020-09-27 23:36:59,差评 辣白菜 五花肉 石锅 拌 饭 超级 难吃 五花肉 硬 难吃 感觉 新鲜 ...
17951,1,146739,15687,2020-09-27 23:53:15,贵 好吃 老婆 几次 非要 一吃 真 难吃 蒸 没错 炒菜 油腻 菜 好...


In [7]:
text=jieba.lcut('新鲜 差评 辣白菜 难吃 五花肉 石锅 硬 拌 饭 超级 感觉')
print(text)

['新鲜', ' ', '差评', ' ', '辣白菜', ' ', '难吃', ' ', '五花肉', ' ', '石锅', ' ', '硬', ' ', '拌', ' ', '饭', ' ', '超级', ' ', '感觉']


In [8]:
cut_text=''
for i in data[data['target']==1]['comment']:
    cut_text=cut_text+str(i)
cut_textlist=cut_text.split()

In [9]:
sorted(Counter(cut_textlist).items(),key=lambda x:x[1],reverse=True)[:10]

[('吃', 1789),
 ('真的', 1665),
 ('味道', 986),
 ('说', 831),
 ('小时', 785),
 ('点', 708),
 ('好吃', 668),
 ('难吃', 649),
 ('送', 632),
 ('太', 602)]

In [10]:
plt.figure(figsize=(20,20),dpi=800)
cloud=WordCloud(font_path="./font.ttf",
                background_color='white',
                max_words=1000,
                scale=4)
word_cloud=cloud.generate_from_text(cut_text)
plt.imshow(word_cloud)
plt.axis('off')
plt.savefig('./Q1/Negativity.jpg')
plt.show()

In [11]:
cut_text=''
for i in data[data['target']==0]['comment']:
    cut_text=cut_text+str(i)
cut_textlist=cut_text.split()

In [12]:
sorted(Counter(cut_textlist).items(),key=lambda x:x[1],reverse=True)[:10]

[('不错', 2716),
 ('吃', 2640),
 ('味道', 1915),
 ('好吃', 1352),
 ('喜欢', 1109),
 ('环境', 757),
 ('感觉', 712),
 ('价格', 629),
 ('挺', 595),
 ('服务', 556)]

In [13]:
plt.figure(figsize=(20,20),dpi=800)
cloud=WordCloud(font_path="./font.ttf",
                background_color='white',
                max_words=1000,
                scale=4)
word_cloud=cloud.generate_from_text(cut_text)
plt.imshow(word_cloud)
plt.axis('off')
plt.savefig('./Q1/acclaim.jpg')
plt.show()